In [21]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf

from os import makedirs
from numpy import zeros
from numpy import ones
from numpy import expand_dims
from numpy import hstack
from numpy.random import randn
from numpy.random import randint
# from tensorflow.keras.datasets.mnist import load_data
from VisionEngine.data import butterflies
from tensorflow.keras import backend
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.constraints import Constraint
from matplotlib import pyplot
from tensorflow.keras.optimizers import RMSprop
import tensorflow.keras.backend as K
from tensorflow.keras.layers import concatenate

In [22]:
tf.__version__

'2.0.0'

In [23]:
tf.executing_eagerly()


True

In [24]:
# from sklearn.model_selection import train_test_split

# x_train, x_test, y_train, y_test = train_test_split(images,lines,test_size=0)

In [25]:
# x_train.shape

In [26]:
# np.savez('guppy_ornaments', x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

In [27]:
# npz = np.load('guppy_ornaments.npz')
# npz.files

In [42]:

# # create a unique directory name for this script
# def get_dir():
#     import sys
#     # get script path
#     arg0 = sys.argv[0]
#     # get filename
#     filename = arg0.split('/')[-1]
#     # remove extension
#     filename = filename[:-3]
#     return filename

# # define the standalone discriminator model
# def define_discriminator(n_cat, n_con, in_shape=(256,256,3)):
#     # weight initialization
#     init = RandomNormal(stddev=0.02)
#     # image input
#     in_image = Input(shape=in_shape)

#     # 128x128
#     d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
#     d = LeakyReLU(alpha=0.1)(d)
#     # 64x64
#     d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
#     d = LeakyReLU(alpha=0.1)(d)
#     # 32x32
#     d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
#     d = LeakyReLU(alpha=0.1)(d)
#     # 16x16
#     d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
#     d = LeakyReLU(alpha=0.1)(d)
#     d = BatchNormalization()(d)
#     # normal
#     d = Conv2D(256, (4,4), padding='same', kernel_initializer=init)(d)
#     d = LeakyReLU(alpha=0.1)(d)
#     d = BatchNormalization()(d)

#     # flatten feature maps
#     d = Flatten()(d)
#     # real/fake output
#     out_classifier = Dense(1, activation='sigmoid')(d)
#     # define d model
#     d_model = Model(in_image, out_classifier)
#     # compile d model
#     d_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
#     # create q model layers
#     q = Dense(128)(d)
#     q = BatchNormalization()(q)
#     q = LeakyReLU(alpha=0.1)(q)
#     # q model output 1
#     out_codes1 = Dense(n_cat, activation='softmax')(q)
#     # q model output 2
#     out_codes2 = Dense(n_con, activation='linear')(q)
#     # define q model
#     q_model = Model(in_image, [out_codes1, out_codes2])
#     return d_model, q_model

# # define the standalone generator model
# def define_generator(gen_input_size):
#     # weight initialization
#     init = RandomNormal(stddev=0.02)
#     # image generator input
#     in_lat = Input(shape=(gen_input_size,))
#     # foundation for 16x16 image
#     n_nodes = 512 * 16 * 16
#     gen = Dense(n_nodes, kernel_initializer=init)(in_lat)
#     gen = Activation('relu')(gen)
#     gen = BatchNormalization()(gen)
#     gen = Reshape((16, 16, 512))(gen)

#     # normal
#     gen = Conv2D(128, (4,4), padding='same', kernel_initializer=init)(gen)
#     gen = Activation('relu')(gen)
#     gen = BatchNormalization()(gen)
#     # 32x32
#     gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
#     gen = Activation('relu')(gen)
#     gen = BatchNormalization()(gen)
#     # 64x64
#     gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
#     gen = Activation('relu')(gen)
#     gen = BatchNormalization()(gen)
#     # 128x128
#     gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
#     gen = Activation('relu')(gen)
#     gen = BatchNormalization()(gen)
#     # 256x256
#     gen = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)

#     # tanh output
#     out_layer = Activation('tanh')(gen)
#     # define model
#     model = Model(in_lat, out_layer)
#     return model

# # define the combined discriminator, generator and q network model
# def define_gan(g_model, d_model, q_model):
#     # make weights in the discriminator (some shared with the q model) as not trainable
#     d_model.trainable = False
#     # connect g outputs to d inputs
#     d_output = d_model(g_model.output)
#     # connect g outputs to q inputs
#     q_output = q_model(g_model.output)
#     # define composite model
#     model = Model(g_model.input, [d_output]+q_output)
#     # compile model
#     opt = Adam(lr=0.0002, beta_1=0.5)
#     model.compile(loss=['binary_crossentropy', 'categorical_crossentropy', 'mse'], optimizer=opt)
#     d_model.compile(opt, loss='binary_crossentropy')
#     return model

# # load images
# def load_real_samples():
#     # load dataset
#     (trainX, _), (_, _) = load_data()
#     # expand to 3d, e.g. add channels
# #     X = expand_dims(trainX, axis=-1)
#     # convert from ints to floats
#     X = trainX.astype('float32')
#     # scale from [0,255] to [-1,1]
#     X = (X - 127.5) / 127.5
#     print(X.shape)
#     return X

# # select real samples
# def generate_real_samples(dataset, n_samples):
#     # choose random instances
#     ix = randint(0, dataset.shape[0], n_samples)
#     # select images and labels
#     X = dataset[ix]
#     # generate class labels
#     y = ones((n_samples, 1))
#     return X, y

# # generate points in latent space as input for the generator
# def generate_latent_points(latent_dim, n_cat, n_con, n_samples):
#     # generate points in the latent space
#     z_latent = randn(latent_dim * n_samples)
#     # reshape into a batch of inputs for the network
#     z_latent = z_latent.reshape(n_samples, latent_dim)
#     # generate categorical codes
#     cat_codes = randint(0, n_cat, n_samples)
#     # one hot encode
#     cat_codes = to_categorical(cat_codes, num_classes=n_cat)
#     # generate continuous codes
#     con_codes = randn(n_con * n_samples)
#     # reshape continuous codes
#     con_codes = con_codes.reshape(n_samples, n_con)
#     # concatenate latent points and control codes
#     z_input = hstack((z_latent, cat_codes, con_codes))
#     return [z_input, cat_codes, con_codes]

# # use the generator to generate n fake examples, with class labels
# def generate_fake_samples(generator, latent_dim, n_cat, n_con, n_samples):
#     # generate points in latent space and control codes
#     z_input, _, _ = generate_latent_points(latent_dim, n_cat, n_con, n_samples)
#     # predict outputs
#     images = generator.predict(z_input)
#     # create class labels
#     y = zeros((n_samples, 1))
#     return images, y

# # generate samples and save as a plot and save the model
# def summarize_performance(step, g_model, gan_model, latent_dim, n_cat, n_con, dataset, n_samples=100):
#     # prepare fake examples
#     X, _ = generate_fake_samples(g_model, latent_dim, n_cat, n_con, n_samples)
#     # scale from [-1,1] to [0,1]
#     X = (X + 1) / 2.0
#     # plot images
#     for i in range(100):
#         # define subplot
#         pyplot.subplot(10, 10, 1 + i)
#         # turn off axis
#         pyplot.axis('off')
#         # plot raw pixel data
#         pyplot.imshow(X[i, :, :, :])
#     # save plot to file
#     filename1 = '%s/generated_plot_%04d.png' % (get_dir(), (step+1))
#     pyplot.savefig(filename1)
#     pyplot.close()
# #     Y, _ = generate_real_samples(dataset, n_samples)
# #     for i in range(100):
# #         # define subplot
# #         pyplot.subplot(10, 10, 1 + i)
# #         # turn off axis
# #         pyplot.axis('off')
# #         # plot raw pixel data
# #         pyplot.imshow(Y[i, :, :, :])
# #     # save plot to file
# #     filename1 = '%s/real_plot_%04d.png' % (get_dir(), (step+1))
# #     pyplot.savefig(filename1)
# #     pyplot.close()
#     # save the generator model
#     filename2 = '%s/model_%04d.h5' % (get_dir(), (step+1))
#     g_model.save(filename2)
#     # save the gan model
#     filename3 = '%s/gan_model_%04d.h5' % (get_dir(), (step+1))
#     gan_model.save(filename3)
#     print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))

# # train the generator and discriminator
# def train(g_model, d_model, gan_model, dataset, latent_dim, n_cat, n_con, n_epochs=5000, n_batch=64):
#     # calculate the number of batches per training epoch
#     bat_per_epo = int(dataset.shape[0] / n_batch)
#     # calculate the number of training iterations
#     n_steps = bat_per_epo * n_epochs
#     # calculate the size of half a batch of samples
#     half_batch = int(n_batch / 2)
#     # manually enumerate epochs
#     for i in range(n_steps):
#         # get randomly selected 'real' samples
#         X_real, y_real = generate_real_samples(dataset, half_batch)
#         # update discriminator and q model weights
#         d_loss1 = d_model.train_on_batch(X_real, y_real)
#         # generate 'fake' examples
#         X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_cat, n_con, half_batch)
#         # update discriminator model weights
#         d_loss2 = d_model.train_on_batch(X_fake, y_fake)
#         # prepare points in latent space as input for the generator
#         z_input, cat_codes, con_codes = generate_latent_points(latent_dim, n_cat, n_con, n_batch)
#         # create inverted labels for the fake samples
#         y_gan = ones((n_batch, 1))
#         # update the g via the d and q error
#         _,g_1,g_2,g3 = gan_model.train_on_batch(z_input, [y_gan, cat_codes, con_codes])
#         # summarize loss on this batch
#         # evaluate the model performance every 'epoch'
#         if (i+1) % (bat_per_epo * 100) == 0:
#             print('>%d, d[%.3f,%.3f], g[%.3f] q1[%.3f], q2[%.3f]' % (i+1, d_loss1, d_loss2, g_1, g_2, g3))
#             summarize_performance(i, g_model, gan_model, latent_dim, n_cat, n_con, dataset)

In [43]:
# # make folder for images
# makedirs(get_dir(), exist_ok=True)
# # number of categorical control codes
# n_cat = 13
# # number of continuous control codes
# n_con = 32
# # size of the latent space
# latent_dim = 100
# # create the discriminator
# d_model, q_model = define_discriminator(n_cat, n_con)
# # create the generator
# gen_input_size = latent_dim + n_cat + n_con
# g_model = define_generator(gen_input_size)
# # create the gan
# gan_model = define_gan(g_model, d_model, q_model)
# # load image data
# dataset = load_real_samples()
# # train model
# train(g_model, d_model, gan_model, dataset, latent_dim, n_cat, n_con)

In [44]:
# # # define the standalone discriminator model
# # def define_discriminator(n_cat, in_shape=(28,28,1)):
# #     # weight initialization
# #     init = RandomNormal(stddev=0.02)
# #     # image input
# #     in_image = Input(shape=in_shape)
# #     # downsample to 14x14
# #     d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
# #     d = LeakyReLU(alpha=0.1)(d)
# #     # downsample to 7x7
# #     d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
# #     d = LeakyReLU(alpha=0.1)(d)
# #     d = BatchNormalization()(d)
# #     # normal
# #     d = Conv2D(256, (4,4), padding='same', kernel_initializer=init)(d)
# #     d = LeakyReLU(alpha=0.1)(d)
# #     d = BatchNormalization()(d)
# #     # flatten feature maps
# #     d = Flatten()(d)
# #     # real/fake output
# #     out_classifier = Dense(1, activation='sigmoid')(d)
# #     # define d model
# #     d_model = Model(in_image, out_classifier)
# #     # compile d model
# #     d_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
# #     # create q model layers
# #     q = Dense(128)(d)
# #     q = BatchNormalization()(q)
# #     q = LeakyReLU(alpha=0.1)(q)
# #     # q model output
# #     out_codes = Dense(n_cat, activation='softmax')(q)
# #     # define q model
# #     q_model = Model(in_image, out_codes)
# #     return d_model, q_model

# def wasserstein_loss(y_true, y_pred):
#     return K.mean(y_true * y_pred)

# # define the standalone discriminator model
# def define_discriminator(n_cat, in_shape=(256,256,3)):
#     # weight initialization
#     init = RandomNormal(stddev=0.02)
#     # image input
#     in_image = Input(shape=in_shape)

#     # 128x128
#     d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
#     d = LeakyReLU(alpha=0.1)(d)
#     # 64x64
#     d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
#     d = LeakyReLU(alpha=0.1)(d)
#     # 32x32
#     d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
#     d = LeakyReLU(alpha=0.1)(d)
#     # 16x16
#     d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
#     d = LeakyReLU(alpha=0.1)(d)
#     d = BatchNormalization()(d)
#     # normal
#     d = Conv2D(256, (4,4), padding='same', kernel_initializer=init)(d)
#     d = LeakyReLU(alpha=0.1)(d)
#     d = BatchNormalization()(d)

#     # flatten feature maps
#     d = Flatten()(d)
#     # real/fake output
#     out_classifier = Dense(1, activation='sigmoid')(d)
#     # define d model
#     d_model = Model(in_image, out_classifier)
#     # compile d model
#     d_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
#     # create q model layers
#     q = Dense(128)(d)
#     q = BatchNormalization()(q)
#     q = LeakyReLU(alpha=0.1)(q)
#     # q model output 1
#     out_codes = Dense(n_cat, activation='softmax')(q)
#     # define q model
#     q_model = Model(in_image, out_codes)
#     return d_model, q_model

# # # define the standalone generator model
# #     # weight initialization
# #     init = RandomNormal(stddev=0.02)
# #     # image generator input
# #     in_lat = Input(shape=(gen_input_size,))
# #     # foundation for 7x7 image
# #     n_nodes = 512 * 7 * 7
# #     gen = Dense(n_nodes, kernel_initializer=init)(in_lat)
# #     gen = Activation('relu')(gen)
# #     gen = BatchNormalization()(gen)
# #     gen = Reshape((7, 7, 512))(gen)
# #     # normal
# #     gen = Conv2D(128, (4,4), padding='same', kernel_initializer=init)(gen)
# #     gen = Activation('relu')(gen)
# #     gen = BatchNormalization()(gen)
# #     # upsample to 14x14
# #     gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
# #     gen = Activation('relu')(gen)
# #     gen = BatchNormalization()(gen)
# #     # upsample to 28x28
# #     gen = Conv2DTranspose(1, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
# #     # tanh output
# #     out_layer = Activation('tanh')(gen)
# #     # define model
# #     model = Model(in_lat, out_layer)
# #     return model

# # define the standalone generator model
# def define_generator(gen_input_size):
#     # weight initialization
#     init = RandomNormal(stddev=0.02)
#     # image generator input
#     in_lat = Input(shape=(gen_input_size,))
#     # foundation for 16x16 image
#     n_nodes = 512 * 16 * 16
#     gen = Dense(n_nodes, kernel_initializer=init)(in_lat)
#     gen = Activation('relu')(gen)
#     gen = BatchNormalization()(gen)
#     gen = Reshape((16, 16, 512))(gen)

#     # normal
#     gen = Conv2D(128, (4,4), padding='same', kernel_initializer=init)(gen)
#     gen = Activation('relu')(gen)
#     gen = BatchNormalization()(gen)
#     # 32x32
#     gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
#     gen = Activation('relu')(gen)
#     gen = BatchNormalization()(gen)
#     # 64x64
#     gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
#     gen = Activation('relu')(gen)
#     gen = BatchNormalization()(gen)
#     # 128x128
#     gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
#     gen = Activation('relu')(gen)
#     gen = BatchNormalization()(gen)
#     # 256x256
#     gen = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)

#     # tanh output
#     out_layer = Activation('tanh')(gen)
#     # define model
#     model = Model(in_lat, out_layer)
#     return model

# # define the combined discriminator, generator and q network model
# def define_gan(g_model, d_model, q_model):
#     # make weights in the discriminator (some shared with the q model) as not trainable
#     d_model.trainable = False
#     # connect g outputs to d inputs
#     d_output = d_model(g_model.output)
#     # connect g outputs to q inputs
#     q_output = q_model(g_model.output)
#     # define composite model
#     model = Model(g_model.input, [d_output, q_output])
#     # compile model
#     opt = Adam()
#     model.compile(loss=['binary_crossentropy', 'categorical_crossentropy'], optimizer=opt)
#     d_model.compile(opt, loss='binary_crossentropy')
#     return model

# # load images
# def load_real_samples():
#     # load dataset
#     (trainX, _), (_, _) = load_data()
#     # expand to 3d, e.g. add channels
# #     X = expand_dims(trainX, axis=-1)
#     # convert from ints to floats
#     X = trainX.astype('float32')
#     # scale from [0,255] to [-1,1]
# #     X = (X - 127.5) / 127.5
#     print(X.shape)
#     return X

# # select real samples
# def generate_real_samples(dataset, n_samples):
#     # choose random instances
#     ix = randint(0, dataset.shape[0], n_samples)
#     # select images and labels
#     X = dataset[ix]
#     # generate class labels
#     y = ones((n_samples, 1))
#     return X, y

# # generate points in latent space as input for the generator
# def generate_latent_points(latent_dim, n_cat, n_samples):
#     # generate points in the latent space
#     z_latent = randn(latent_dim * n_samples)
#     # reshape into a batch of inputs for the network
#     z_latent = z_latent.reshape(n_samples, latent_dim)
#     # generate categorical codes
#     cat_codes = randint(0, n_cat, n_samples)
#     # one hot encode
#     cat_codes = to_categorical(cat_codes, num_classes=n_cat)
#     # concatenate latent points and control codes
#     z_input = hstack((z_latent, cat_codes))
#     return [z_input, cat_codes]

# # use the generator to generate n fake examples, with class labels
# def generate_fake_samples(generator, latent_dim, n_cat, n_samples):
#     # generate points in latent space and control codes
#     z_input, _ = generate_latent_points(latent_dim, n_cat, n_samples)
#     # predict outputs
#     images = generator.predict(z_input)
#     # create class labels
#     y = zeros((n_samples, 1))
#     return images, y

# # generate samples and save as a plot and save the model
# def summarize_performance(step, g_model, gan_model, latent_dim, n_cat, n_samples=100):
#     # prepare fake examples
#     X, _ = generate_fake_samples(g_model, latent_dim, n_cat, n_samples)
#     X = (X + 1) / 2.0
#     # plot images
#     for i in range(100):
#         # define subplot
#         pyplot.subplot(10, 10, 1 + i)
#         # turn off axis
#         pyplot.axis('off')
#         # plot raw pixel data
#         pyplot.imshow(X[i, :, :, :])
#     # save plot to file
#     filename1 = '%s/generated_plot_%04d.png' % (get_dir(), (step+1))
#     pyplot.savefig(filename1)
#     pyplot.close()
#     # save the generator model
#     filename2 = 'model_%04d.h5' % (step+1)
#     g_model.save(filename2)
#     # save the gan model
#     filename3 = 'gan_model_%04d.h5' % (step+1)
#     gan_model.save(filename3)
#     print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))

# # train the generator and discriminator
# def train(g_model, d_model, gan_model, dataset, latent_dim, n_cat, n_epochs=5000, n_batch=32):
#     # calculate the number of batches per training epoch
#     bat_per_epo = int(dataset.shape[0] / n_batch)
#     # calculate the number of training iterations
#     n_steps = bat_per_epo * n_epochs
#     # calculate the size of half a batch of samples
#     half_batch = int(n_batch / 2)
#     # manually enumerate epochs
#     for i in range(n_steps):
#         # get randomly selected 'real' samples
#         X_real, y_real = generate_real_samples(dataset, half_batch)
#         # update discriminator and q model weights
#         d_loss1 = d_model.train_on_batch(X_real, y_real)
#         # generate 'fake' examples
#         X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_cat, half_batch)
#         # update discriminator model weights
#         d_loss2 = d_model.train_on_batch(X_fake, y_fake)
#         # prepare points in latent space as input for the generator
#         z_input, cat_codes = generate_latent_points(latent_dim, n_cat, n_batch)
#         # create inverted labels for the fake samples
#         y_gan = ones((n_batch, 1))
#         # update the g via the d and q error
#         _,g_1,g_2 = gan_model.train_on_batch(z_input, [y_gan, cat_codes])
#         # summarize loss on this batch
        
#         # evaluate the model performance every 'epoch'
#         if (i+1) % (bat_per_epo * 100) == 0:
#             print('>%d, d[%.3f,%.3f], g[%.3f] q[%.3f]' % (i+1, d_loss1, d_loss2, g_1, g_2))
#             summarize_performance(i, g_model, gan_model, latent_dim, n_cat)



In [45]:
# makedirs(get_dir(), exist_ok=True)
# # number of values for the categorical control code
# n_cat = 13
# # size of the latent space
# latent_dim = 100
# # create the discriminator
# d_model, q_model = define_discriminator(n_cat)
# # create the generator
# gen_input_size = latent_dim + n_cat
# g_model = define_generator(gen_input_size)
# # create the gan
# gan_model = define_gan(g_model, d_model, q_model)
# # load image data
# dataset = load_real_samples()
# # train model
# train(g_model, d_model, gan_model, dataset, latent_dim, n_cat)

In [46]:


# from keras.layers import Activation, Dense, Input
# from keras.layers import Conv2D, Flatten
# from keras.layers import Reshape, Conv2DTranspose
# from keras.layers import LeakyReLU
# from keras.layers import BatchNormalization
# from keras.models import Model
# from keras.layers.merge import concatenate

# import numpy as np
# import math
# import matplotlib.pyplot as plt
# import os

# def generator(inputs,
#               image_size,
#               activation='sigmoid',
#               labels=None,
#               codes=None):
#     """Build a Generator Model
#     Stack of BN-ReLU-Conv2DTranpose to generate fake images.
#     Output activation is sigmoid instead of tanh in [1].
#     Sigmoid converges easily.
#     # Arguments
#         inputs (Layer): Input layer of the generator (the z-vector)
#         image_size (int): Target size of one side (assuming square image)
#         activation (string): Name of output activation layer
#         labels (tensor): Input labels
#         codes (list): 2-dim disentangled codes for InfoGAN
#     # Returns
#         Model: Generator Model
#     """
#     image_resize = image_size // 4
#     # network parameters
#     kernel_size = 5
#     layer_filters = [128, 64, 32, 3]

#     if labels is not None:
#         if codes is None:
#             # ACGAN labels
#             # concatenate z noise vector and one-hot labels
#             inputs = [inputs, labels]
#         else:
#             # infoGAN codes
#             # concatenate z noise vector, one-hot labels and codes 1 & 2
#             inputs = [inputs, labels] + codes
#         x = concatenate(inputs, axis=1)
#     elif codes is not None:
#         # generator 0 of StackedGAN
#         inputs = [inputs, codes]
#         x = concatenate(inputs, axis=1)
#     else:
#         # default input is just 100-dim noise (z-code)
#         x = inputs

#     x = Dense(image_resize * image_resize * layer_filters[0])(x)
#     x = Reshape((image_resize, image_resize, layer_filters[0]))(x)

#     for filters in layer_filters:
#         # first two convolution layers use strides = 2
#         # the last two use strides = 1
#         if filters > layer_filters[-2]:
#             strides = 2
#         else:
#             strides = 1
#         x = BatchNormalization()(x)
#         x = Activation('relu')(x)
#         x = Conv2DTranspose(filters=filters,
#                             kernel_size=kernel_size,
#                             strides=strides,
#                             padding='same')(x)

#     if activation is not None:
#         x = Activation(activation)(x)

#     # generator output is the synthesized image x
#     return Model(inputs, x, name='generator')


# def discriminator(inputs,
#                   activation='sigmoid',
#                   num_labels=13,
#                   num_codes=2):
#     """Build a Discriminator Model
#     Stack of LeakyReLU-Conv2D to discriminate real from fake
#     The network does not converge with BN so it is not used here
#     unlike in [1]
#     # Arguments
#         inputs (Layer): Input layer of the discriminator (the image)
#         activation (string): Name of output activation layer
#         num_labels (int): Dimension of one-hot labels for ACGAN & InfoGAN
#         num_codes (int): num_codes-dim Q network as output 
#                     if StackedGAN or 2 Q networks if InfoGAN
                    
#     # Returns
#         Model: Discriminator Model
#     """
#     kernel_size = 5
#     layer_filters = [32, 64, 128, 256]

#     x = inputs
#     for filters in layer_filters:
#         # first 3 convolution layers use strides = 2
#         # last one uses strides = 1
#         if filters == layer_filters[-1]:
#             strides = 1
#         else:
#             strides = 2
        
#         x = Conv2D(filters=filters,
#                    kernel_size=kernel_size,
#                    strides=strides,
#                    padding='same')(x)
#         x = LeakyReLU(alpha=0.2)(x)

#     x = Flatten()(x)
#     # default output is probability that the image is real
#     outputs = Dense(1)(x)
#     if activation is not None:
#         print(activation)
#         outputs = Activation(activation)(outputs)

#     if num_labels:
#         # ACGAN and InfoGAN have 2nd output
#         # 2nd output is 10-dim one-hot vector of label
#         layer = Dense(layer_filters[-2])(x)
#         labels = Dense(num_labels)(layer)
#         labels = Activation('softmax', name='label')(labels)
#         if num_codes is None:
#             outputs = [outputs, labels]
#         else:
#             # InfoGAN have 3rd and 4th outputs
#             # 3rd output is 1-dim continous Q of 1st c given x
#             code1 = Dense(1)(layer)
#             code1 = Activation('sigmoid', name='code1')(code1)

#             # 4th output is 1-dim continuous Q of 2nd c given x
#             code2 = Dense(1)(layer)
#             code2 = Activation('sigmoid', name='code2')(code2)

#             outputs = [outputs, labels, code1, code2]
#     elif num_codes is not None:
#         # StackedGAN Q0 output
#         # z0_recon is reconstruction of z0 normal distribution
#         z0_recon =  Dense(num_codes)(x)
#         z0_recon = Activation('tanh', name='z0')(z0_recon)
#         outputs = [outputs, z0_recon]

#     return Model(inputs, outputs, name='discriminator')


# def train(models, x_train, params):
#     """Train the Discriminator and Adversarial Networks
#     Alternately train Discriminator and Adversarial networks by batch.
#     Discriminator is trained first with properly real and fake images.
#     Adversarial is trained next with fake images pretending to be real
#     Generate sample images per save_interval.
#     # Arguments
#         models (list): Generator, Discriminator, Adversarial models
#         x_train (tensor): Train images
#         params (list) : Networks parameters
#     """
#     # the GAN models
#     generator, discriminator, adversarial = models
#     # network parameters
#     batch_size, latent_size, train_steps, model_name = params
#     # the generator image is saved every 500 steps
#     save_interval = 500
#     # noise vector to see how the generator output evolves during training
#     noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
#     # number of elements in train dataset
#     train_size = x_train.shape[0]
#     for i in range(train_steps):
#         # train the discriminator for 1 batch
#         # 1 batch of real (label=1.0) and fake images (label=0.0)
#         # randomly pick real images from dataset
#         rand_indexes = np.random.randint(0, train_size, size=batch_size)
#         real_images = x_train[rand_indexes]
#         # generate fake images from noise using generator 
#         # generate noise using uniform distribution
#         noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
#         # generate fake images
#         fake_images = generator.predict(noise)
#         # real + fake images = 1 batch of train data
#         x = np.concatenate((real_images, fake_images))
#         # label real and fake images
#         # real images label is 1.0
#         y = np.ones([2 * batch_size, 1])
#         # fake images label is 0.0
#         y[batch_size:, :] = 0.0
#         # train discriminator network, log the loss and accuracy
#         loss, acc = discriminator.train_on_batch(x, y)
#         log = "%d: [discriminator loss: %f, acc: %f]" % (i, loss, acc)

#         # train the adversarial network for 1 batch
#         # 1 batch of fake images with label=1.0
#         # since the discriminator weights are frozen in adversarial network
#         # only the generator is trained
#         # generate noise using uniform distribution
#         noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
#         # label fake images as real or 1.0
#         y = np.ones([batch_size, 1])
#         # train the adversarial network 
#         # note that unlike in discriminator training, 
#         # we do not save the fake images in a variable
#         # the fake images go to the discriminator input of the adversarial
#         # for classification
#         # log the loss and accuracy
#         loss, acc = adversarial.train_on_batch(noise, y)
#         log = "%s [adversarial loss: %f, acc: %f]" % (log, loss, acc)
#         print(log)
#         if (i + 1) % save_interval == 0:
#             if (i + 1) == train_steps:
#                 show = True
#             else:
#                 show = False

#             # plot generator images on a periodic basis
#             plot_images(generator,
#                         noise_input=noise_input,
#                         show=show,
#                         step=(i + 1),
#                         model_name=model_name)
   
#     # save the model after training the generator
#     # the trained generator can be reloaded for future MNIST digit generation
#     generator.save(model_name + ".h5")


# def plot_images(generator,
#                 noise_input,
#                 noise_label=None,
#                 noise_codes=None,
#                 show=False,
#                 step=0,
#                 model_name="gan"):
#     """Generate fake images and plot them
#     For visualization purposes, generate fake images
#     then plot them in a square grid
#     # Arguments
#         generator (Model): The Generator Model for fake images generation
#         noise_input (ndarray): Array of z-vectors
#         show (bool): Whether to show plot or not
#         step (int): Appended to filename of the save images
#         model_name (string): Model name
#     """
#     os.makedirs(model_name, exist_ok=True)
#     filename = os.path.join(model_name, "%05d.png" % step)
#     rows = int(math.sqrt(noise_input.shape[0]))
#     if noise_label is not None:
#         noise_input = [noise_input, noise_label]
#         if noise_codes is not None:
#             noise_input += noise_codes

#     images = generator.predict(noise_input)
#     plt.figure(figsize=(10, 10))
#     num_images = images.shape[0]
#     image_size = images.shape[1]
#     for i in range(num_images):
#         plt.subplot(rows, rows, i + 1)
#         image = np.reshape(images[i], [image_size, image_size, 3])
#         plt.imshow(image)
#         plt.axis('off')
#     plt.savefig(filename)
#     if show:
#         plt.show()
#     else:
#         plt.close('all')


# def test_generator(generator):
#     noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
#     plot_images(generator,
#                 noise_input=noise_input,
#                 show=True,
#                 model_name="test_outputs")

In [47]:
# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function

# from keras.layers import Input
# from keras.optimizers import RMSprop
# from keras.models import Model
# from keras.datasets import mnist
# from keras.utils import to_categorical
# from keras.models import load_model
# from keras import backend as K

# import numpy as np
# import argparse

# import sys
# sys.path.append("..")


# # from ..lib import gan

# def train(models, data, params):
#     """Train the Discriminator and Adversarial networks
#     Alternately train discriminator and adversarial networks by batch.
#     Discriminator is trained first with real and fake images,
#     corresponding one-hot labels and continuous codes.
#     Adversarial is trained next with fake images pretending to be real,
#     corresponding one-hot labels and continous codes.
#     Generate sample images per save_interval.
#     # Arguments
#         models (Models): Generator, Discriminator, Adversarial models
#         data (tuple): x_train, y_train data
#         params (tuple): Network parameters
#     """
#     # the GAN models
#     generator, discriminator, adversarial = models
#     # images and their one-hot labels
#     x_train, y_train = data
#     # network parameters
#     batch_size, latent_size, train_steps, num_labels, model_name = params
#     # the generator image is saved every 500 steps
#     save_interval = 500
#     # noise vector to see how the generator output evolves during training
#     noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
#     # random class labels and codes
#     noise_label = np.eye(num_labels)[np.arange(0, 16) % num_labels]
#     noise_code1 = np.random.normal(scale=0.5, size=[16, 1])
#     noise_code2 = np.random.normal(scale=0.5, size=[16, 1])
#     # number of elements in train dataset
#     train_size = x_train.shape[0]
#     print(model_name,
#           "Labels for generated images: ",
#           np.argmax(noise_label, axis=1))

#     for i in range(train_steps):
#         # train the discriminator for 1 batch
#         # 1 batch of real (label=1.0) and fake images (label=0.0)
#         # randomly pick real images and corresponding labels from dataset 
#         rand_indexes = np.random.randint(0, train_size, size=batch_size)
#         real_images = x_train[rand_indexes]
#         real_labels = y_train[rand_indexes]
#         # random codes for real images
#         real_code1 = np.random.normal(scale=0.5, size=[batch_size, 1])
#         real_code2 = np.random.normal(scale=0.5, size=[batch_size, 1])
#         # generate fake images, labels and codes
#         noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
#         fake_labels = np.eye(num_labels)[np.random.choice(num_labels,
#                                                           batch_size)]
#         fake_code1 = np.random.normal(scale=0.5, size=[batch_size, 1])
#         fake_code2 = np.random.normal(scale=0.5, size=[batch_size, 1])
#         inputs = [noise, fake_labels, fake_code1, fake_code2]
#         fake_images = generator.predict(inputs)

#         # real + fake images = 1 batch of train data
#         x = np.concatenate((real_images, fake_images))
#         labels = np.concatenate((real_labels, fake_labels))
#         codes1 = np.concatenate((real_code1, fake_code1))
#         codes2 = np.concatenate((real_code2, fake_code2))

#         # label real and fake images
#         # real images label is 1.0
#         y = np.ones([2 * batch_size, 1])
#         # fake images label is 0.0
#         y[batch_size:, :] = 0

#         # train discriminator network, log the loss and label accuracy
#         outputs = [y, labels, codes1, codes2]
#         # metrics = ['loss', 'activation_1_loss', 'label_loss',
#         # 'code1_loss', 'code2_loss', 'activation_1_acc',
#         # 'label_acc', 'code1_acc', 'code2_acc']
#         # from discriminator.metrics_names
#         metrics = discriminator.train_on_batch(x, outputs)
#         fmt = "%d: [discriminator loss: %f, label_acc: %f]"
#         log = fmt % (i, metrics[0], metrics[6])

#         # train the adversarial network for 1 batch
#         # 1 batch of fake images with label=1.0 and
#         # corresponding one-hot label or class + random codes
#         # since the discriminator weights are frozen in adversarial network
#         # only the generator is trained
#         # generate fake images, labels and codes
#         noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
#         fake_labels = np.eye(num_labels)[np.random.choice(num_labels,
#                                                           batch_size)]
#         fake_code1 = np.random.normal(scale=0.5, size=[batch_size, 1])
#         fake_code2 = np.random.normal(scale=0.5, size=[batch_size, 1])
#         # label fake images as real
#         y = np.ones([batch_size, 1])

#         # train the adversarial network 
#         # note that unlike in discriminator training, 
#         # we do not save the fake images in a variable
#         # the fake images go to the discriminator input of the adversarial
#         # for classification
#         # log the loss and label accuracy
#         inputs = [noise, fake_labels, fake_code1, fake_code2]
#         outputs = [y, fake_labels, fake_code1, fake_code2]
#         metrics  = adversarial.train_on_batch(inputs, outputs)
#         fmt = "%s [adversarial loss: %f, label_acc: %f]"
#         log = fmt % (log, metrics[0], metrics[6])

#         print(log)
#         if (i + 1) % save_interval == 0:
#             if (i + 1) == train_steps:
#                 show = True
#             else:
#                 show = False
    
#             # plot generator images on a periodic basis
#             plot_images(generator,
#                             noise_input=noise_input,
#                             noise_label=noise_label,
#                             noise_codes=[noise_code1, noise_code2],
#                             show=show,
#                             step=(i + 1),
#                             model_name=model_name)
   
#     # save the model after training the generator
#     # the trained generator can be reloaded for
#     # future MNIST digit generation
#     generator.save(model_name + ".h5")


# def mi_loss(c, q_of_c_given_x):
#     """ Mutual information, Equation 5 in [2] , assuming H(c) is constant"""
#     # mi_loss = -c * log(Q(c|x))
#     return K.mean(-K.sum(K.log(q_of_c_given_x + K.epsilon()) * c, axis=1))


# def test_generator(generator, params, latent_size=100):
#     label, code1, code2, p1, p2 = params
#     noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
#     step = 0
#     if label is None:
#         num_labels = 13
#         noise_label = np.eye(num_labels)[np.random.choice(num_labels, 16)]
#     else:
#         noise_label = np.zeros((16, 13))
#         noise_label[:,label] = 1
#         step = label

#     if code1 is None:
#         noise_code1 = np.random.normal(scale=0.5, size=[16, 1])
#     else:
#         if p1:
#             a = np.linspace(-2, 2, 16)
#             a = np.reshape(a, [16, 1])
#             noise_code1 = np.ones((16, 1)) * a
#         else:
#             noise_code1 = np.ones((16, 1)) * code1
#         print(noise_code1)

#     if code2 is None:
#         noise_code2 = np.random.normal(scale=0.5, size=[16, 1])
#     else:
#         if p2:
#             a = np.linspace(-2, 2, 16)
#             a = np.reshape(a, [16, 1])
#             noise_code2 = np.ones((16, 1)) * a
#         else:
#             noise_code2 = np.ones((16, 1)) * code2
#         print(noise_code2)

#     plot_images(generator,
#                     noise_input=noise_input,
#                     noise_label=noise_label,
#                     noise_codes=[noise_code1, noise_code2],
#                     show=True,
#                     step=step,
#                     model_name="test_outputs")


In [58]:
# model_name = "infogan_guppies"

# # load dataset
# (x_train, y_train), (_, _) = load_data()

# # reshape data for CNN as (28, 28, 1) and normalize
# image_size = x_train.shape[1]
# x_train = np.reshape(x_train, [-1, image_size, image_size, 3])
# x_train = x_train.astype('float32')


# ####
# #plotting sanity check

# os.makedirs(model_name, exist_ok=True)
# filename = os.path.join(model_name, "example_real_inputs.png")
# rows = int(math.sqrt(16))
# images = x_train[0:16]
# plt.figure(figsize=(10, 10))
# num_images = 16
# image_size = 256
# for i in range(num_images):
#     plt.subplot(rows, rows, i + 1)
#     image = np.reshape(images[i], [image_size, image_size, 3])
#     plt.imshow(image)
#     plt.axis('off')
# plt.savefig(filename)
# plt.close('all')
# ####

# # train labels
# num_labels = len(np.unique(y_train))
# print(num_labels)
# _, y_train = np.unique(y_train,return_inverse=True)
# y_train = to_categorical(y_train)

# # network parameters
# batch_size = 32
# train_steps = 40000
# lr = 2e-4
# decay = 6e-8
# input_shape = (image_size, image_size, 3)
# label_shape = (num_labels, )
# code_shape = (1, )
# latent_size = 100

# # build discriminator model
# inputs = Input(shape=input_shape, name='discriminator_input')
# # call discriminator builder with 4 outputs: source, label, and 2 codes
# discriminator = discriminator(inputs,
#                                   num_labels=num_labels,
#                                   num_codes=2)
# # [1] uses Adam, but discriminator converges easily with RMSprop
# optimizer = RMSprop(lr=lr, decay=decay)
# # loss functions: 1) probability image is real (binary crossentropy)
# # 2) categorical cross entropy image label,
# # 3) and 4) mutual information loss
# loss = ['binary_crossentropy', 'categorical_crossentropy', mi_loss, mi_loss]
# # lamda or mi_loss weight is 0.5
# loss_weights = [1.0, 1.0, 0.5, 0.5]
# discriminator.compile(loss=loss,
#                       loss_weights=loss_weights,
#                       optimizer=optimizer,
#                       metrics=['accuracy'])
# discriminator.summary()

# # build generator model
# input_shape = (latent_size, )
# inputs = Input(shape=input_shape, name='z_input')
# labels = Input(shape=label_shape, name='labels')
# code1 = Input(shape=code_shape, name="code1")
# code2 = Input(shape=code_shape, name="code2")
# # call generator with inputs, 
# # labels and codes as total inputs to generator
# generator = generator(inputs,
#                           image_size,
#                           labels=labels,
#                           codes=[code1, code2])
# generator.summary()

# # build adversarial model = generator + discriminator
# optimizer = RMSprop(lr=lr*0.5, decay=decay*0.5)
# discriminator.trainable = False
# # total inputs = noise code, labels, and codes
# inputs = [inputs, labels, code1, code2]
# adversarial = Model(inputs,
#                     discriminator(generator(inputs)),
#                     name=model_name)
# # same loss as discriminator
# adversarial.compile(loss=loss,
#                     loss_weights=loss_weights,
#                     optimizer=optimizer,
#                     metrics=['accuracy'])
# adversarial.summary()

# # train discriminator and adversarial networks
# models = (generator, discriminator, adversarial)
# data = (x_train, y_train)
# params = (batch_size, latent_size, train_steps, num_labels, model_name)
# train(models, data, params)

In [59]:
# generator = load_model("infogan_guppies.h5")
# label = 1
# code1 = -2
# code2 = None
# p1 = False
# p2 = False
# params = (label, code1, code2, p1, p2)
# test_generator(generator, params, latent_size=100)

In [71]:


import numpy as np
import math
import matplotlib.pyplot as plt
import os

def generator(inputs,
              image_size,
              activation='sigmoid',
              labels=None,
              codes=None):
    """Build a Generator Model
    Stack of BN-ReLU-Conv2DTranpose to generate fake images.
    Output activation is sigmoid instead of tanh in [1].
    Sigmoid converges easily.
    # Arguments
        inputs (Layer): Input layer of the generator (the z-vector)
        image_size (int): Target size of one side (assuming square image)
        activation (string): Name of output activation layer
        labels (tensor): Input labels
        codes (list): 2-dim disentangled codes for InfoGAN
    # Returns
        Model: Generator Model
    """
    image_resize = image_size // 4
    # network parameters
    kernel_size = 4
    layer_filters = [128, 64, 32, 3]

    if labels is not None:
        if codes is None:
            # ACGAN labels
            # concatenate z noise vector and one-hot labels
            inputs = [inputs, labels]
        else:
            # infoGAN codes
            # concatenate z noise vector, one-hot labels and codes 1 & 2
            inputs = [inputs, labels] + codes
        x = concatenate(inputs, axis=1)
    elif codes is not None:
        # generator 0 of StackedGAN
        inputs = [inputs, codes]
        x = concatenate(inputs, axis=1)
    else:
        # default input is just 100-dim noise (z-code)
        x = inputs

    x = Dense(image_resize * image_resize * layer_filters[0])(x)
    x = BatchNormalization()(x)
    x = Reshape((image_resize, image_resize, layer_filters[0]))(x)

    for filters in layer_filters:
        # first two convolution layers use strides = 2
        # the last two use strides = 1
        if filters > layer_filters[-2]:
            strides = 2
        else:
            strides = 1
        x = UpSampling2D(size=2)(x)
        x = Conv2D(filters=filters,
                            kernel_size=kernel_size,
                            strides=strides,
                            padding='same')(x)
        x = BatchNormalization(momentum=0.8)(x)
        x = Activation(activation)(x)

#     if activation is not None:
#         x = Activation(activation)(x)

    # generator output is the synthesized image x
    return Model(inputs, x, name='generator')


# def discriminator(inputs,
#                   activation='sigmoid',
#                   num_labels=13,
#                   num_codes=None):
#     """Build a Discriminator Model
#     Stack of LeakyReLU-Conv2D to discriminate real from fake
#     The network does not converge with BN so it is not used here
#     unlike in [1]
#     # Arguments
#         inputs (Layer): Input layer of the discriminator (the image)
#         activation (string): Name of output activation layer
#         num_labels (int): Dimension of one-hot labels for ACGAN & InfoGAN
#         num_codes (int): num_codes-dim Q network as output 
#                     if StackedGAN or 2 Q networks if InfoGAN
                    
#     # Returns
#         Model: Discriminator Model
#     """
#     kernel_size = 4
#     layer_filters = [32, 64, 128, 256]

#     x = inputs
#     for filters in layer_filters:
#         # first 3 convolution layers use strides = 2
#         # last one uses strides = 1
#         if filters == layer_filters[-1]:
#             strides = 1
#         else:
#             strides = 2
        
#         x = Conv2D(filters=filters,
#                    kernel_size=kernel_size,
#                    strides=strides,
#                    padding='same')(x)
#         x = LeakyReLU(alpha=0.2)(x)
#         x = Dropout(0.25)(x)

#     x = Flatten()(x)
#     # default output is probability that the image is real
#     outputs = Dense(1)(x)
#     if activation is not None:
#         print(activation)
#         outputs = Activation(activation)(outputs)

#     if num_labels:
#         layer = Dense(layer_filters[-2])(x)
#         labels = Dense(num_labels)(layer)
#         labels = Activation('softmax', name='label')(labels)
#         if num_codes is None:
#             outputs = [outputs, labels]
#         else:
#             for code_i in range(num_codes+1):
#                 code = Dense(1)(layer)
#                 code = Activation('sigmoid', name='code{}'.format(code_i))(code)
#                 outputs = [outputs, code]
    
#     elif num_codes is not None:

#         z0_recon =  Dense(num_codes)(x)
#         z0_recon = Activation('tanh', name='z0')(z0_recon)
#         outputs = [outputs, z0_recon]
#     outputs = [item for sublist in outputs for item in sublist]
#     return Model(inputs, outputs, name='discriminator')

def discriminator(inputs,
                  activation='sigmoid',
                  num_labels=8,
                  num_codes=None):
    """Build a Discriminator Model
    Stack of LeakyReLU-Conv2D to discriminate real from fake
    The network does not converge with BN so it is not used here
    unlike in [1]
    # Arguments
        inputs (Layer): Input layer of the discriminator (the image)
        activation (string): Name of output activation layer
        num_labels (int): Dimension of one-hot labels for ACGAN & InfoGAN
        num_codes (int): num_codes-dim Q network as output 
                    if StackedGAN or 2 Q networks if InfoGAN
                    
    # Returns
        Model: Discriminator Model
    """
    kernel_size = 4
    layer_filters = [32, 64, 128, 256]

    x = inputs
    for filters in layer_filters:
        # first 3 convolution layers use strides = 2
        # last one uses strides = 1
        if filters == layer_filters[-1]:
            strides = 1
        else:
            strides = 2
        
        x = Conv2D(filters=filters,
                   kernel_size=kernel_size,
                   strides=strides,
                   padding='same')(x)
        x = LeakyReLU(alpha=0.2)(x)
        x = Dropout(0.25)(x)

    x = Flatten()(x)
    # default output is probability that the image is real
    output = Dense(1)(x)
    if activation is not None:
        print(activation)
        output = Activation(activation)(output)
    
    outputs = []
    outputs.append(output)
    if num_labels:
        # ACGAN and InfoGAN have 2nd output
        # 2nd output is 10-dim one-hot vector of label
        layer = Dense(layer_filters[-2])(x)
        labels = Dense(num_labels)(layer)
        labels = Activation('softmax', name='label')(labels)
        outputs.append(labels)
        for code_i in range(num_codes):
        # InfoGAN have 3rd and 4th outputs
        # 3rd output is 1-dim continous Q of 1st c given x
            code = Dense(1)(layer)
            code = Activation('sigmoid', name='code{}'.format(code_i))(code)
            outputs.append(code)
    
    elif num_codes is not None:
        # StackedGAN Q0 output
        # z0_recon is reconstruction of z0 normal distribution
        z0_recon =  Dense(num_codes)(x)
        z0_recon = Activation('tanh', name='z0')(z0_recon)
        outputs.append(z0_recon)
        
    return Model(inputs, outputs, name='discriminator')



def train(models, x_train, params):
    """Train the Discriminator and Adversarial Networks
    Alternately train Discriminator and Adversarial networks by batch.
    Discriminator is trained first with properly real and fake images.
    Adversarial is trained next with fake images pretending to be real
    Generate sample images per save_interval.
    # Arguments
        models (list): Generator, Discriminator, Adversarial models
        x_train (tensor): Train images
        params (list) : Networks parameters
    """
    # the GAN models
    generator, discriminator, adversarial = models
    # network parameters
    batch_size, latent_size, train_steps, model_name = params
    # the generator image is saved every 500 steps
    save_interval = 500
    # noise vector to see how the generator output evolves during training
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
    # number of elements in train dataset
    train_size = x_train.shape[0]
    for i in range(train_steps):
        # train the discriminator for 1 batch
        # 1 batch of real (label=1.0) and fake images (label=0.0)
        # randomly pick real images from dataset
        rand_indexes = np.random.randint(0, train_size, size=batch_size)
        real_images = x_train[rand_indexes]
        # generate fake images from noise using generator 
        # generate noise using uniform distribution
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
        # generate fake images
        fake_images = generator.predict(noise)
        # real + fake images = 1 batch of train data
        x = np.concatenate((real_images, fake_images))
        # label real and fake images
        # real images label is 1.0
        y = np.ones([2 * batch_size, 1])
        # fake images label is 0.0
        y[batch_size:, :] = 0.0
        # train discriminator network, log the loss and accuracy
        loss, acc = discriminator.train_on_batch(x, y)
        log = "%d: [discriminator loss: %f, acc: %f]" % (i, loss, acc)

        # train the adversarial network for 1 batch
        # 1 batch of fake images with label=1.0
        # since the discriminator weights are frozen in adversarial network
        # only the generator is trained
        # generate noise using uniform distribution
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
        # label fake images as real or 1.0
        y = np.ones([batch_size, 1])
        # train the adversarial network 
        # note that unlike in discriminator training, 
        # we do not save the fake images in a variable
        # the fake images go to the discriminator input of the adversarial
        # for classification
        # log the loss and accuracy
        loss, acc = adversarial.train_on_batch(noise, y)
        log = "%s [adversarial loss: %f, acc: %f]" % (log, loss, acc)
        print(log)
        if (i + 1) % save_interval == 0:
            if (i + 1) == train_steps:
                show = True
            else:
                show = False

            # plot generator images on a periodic basis
            plot_images(generator,
                        noise_input=noise_input,
                        show=show,
                        step=(i + 1),
                        model_name=model_name)
   
    # save the model after training the generator
    # the trained generator can be reloaded for future MNIST digit generation
    generator.save(model_name + ".h5")


def plot_images(generator,
                noise_input,
                noise_label=None,
                noise_codes=None,
                show=False,
                step=0,
                model_name="gan"):
    """Generate fake images and plot them
    For visualization purposes, generate fake images
    then plot them in a square grid
    # Arguments
        generator (Model): The Generator Model for fake images generation
        noise_input (ndarray): Array of z-vectors
        show (bool): Whether to show plot or not
        step (int): Appended to filename of the save images
        model_name (string): Model name
    """
    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    rows = int(math.sqrt(noise_input.shape[0]))
    if noise_label is not None:
        noise_input = [noise_input, noise_label]
        if noise_codes is not None:
            noise_input += noise_codes

    images = generator.predict(noise_input)
    plt.figure(figsize=(10, 10))
    num_images = images.shape[0]
    image_size = images.shape[1]
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size, 3])
        plt.imshow(image)
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')

def test_generator(generator):
    noise_input = np.random.uniform(-0.5, 0.5, size=[16, 100])
    plot_images(generator,
                noise_input=noise_input,
                show=True,
                model_name="test_outputs")

In [72]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import numpy as np
import argparse

import sys
sys.path.append("..")


# from ..lib import gan

# def train(models, data, params, num_codes=2):
#     """Train the Discriminator and Adversarial networks
#     Alternately train discriminator and adversarial networks by batch.
#     Discriminator is trained first with real and fake images,
#     corresponding one-hot labels and continuous codes.
#     Adversarial is trained next with fake images pretending to be real,
#     corresponding one-hot labels and continous codes.
#     Generate sample images per save_interval.
#     # Arguments
#         models (Models): Generator, Discriminator, Adversarial models
#         data (tuple): x_train, y_train data
#         params (tuple): Network parameters
#     """
#     # the GAN models
#     generator, discriminator, adversarial = models
#     # images and their one-hot labels
#     x_train, y_train = data
#     # network parameters
#     batch_size, latent_size, train_steps, num_labels, model_name = params
#     print(num_labels)
#     # the generator image is saved every 500 steps
#     save_interval = 100
#     # noise vector to see how the generator output evolves during training
#     noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
#     # random class labels and codes
#     noise_label = np.eye(num_labels)[np.arange(0, 16) % num_labels]
    
#     noise_codes = [np.random.normal(scale=0.5, size=[16, 1]) for i in range(num_codes)]
# #     noise_code1 = np.random.normal(scale=0.5, size=[16, 1])
# #     noise_code2 = np.random.normal(scale=0.5, size=[16, 1])
#     # number of elements in train dataset
#     train_size = x_train.shape[0]
#     print(model_name,
#           "Labels for generated images: ",
#           np.argmax(noise_label, axis=1))

#     for i in range(train_steps):
#         # train the discriminator for 1 batch
#         # 1 batch of real (label=1.0) and fake images (label=0.0)
#         # randomly pick real images and corresponding labels from dataset 
#         rand_indexes = np.random.randint(0, train_size, size=batch_size)
#         real_images = x_train[rand_indexes]
#         real_labels = y_train[rand_indexes]
#         # random codes for real images
#         real_codes = [np.random.normal(scale=0.5, size=[batch_size, 1]) for i in range(num_codes)]
# #         real_code1 = np.random.normal(scale=0.5, size=[batch_size, 1])
# #         real_code2 = np.random.normal(scale=0.5, size=[batch_size, 1])
#         # generate fake images, labels and codes
#         noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
#         fake_labels = np.eye(num_labels)[np.random.choice(num_labels,
#                                                           batch_size)]
#         fake_codes = [np.random.normal(scale=0.5, size=[batch_size, 1]) for i in range(num_codes)]
# #         fake_code1 = np.random.normal(scale=0.5, size=[batch_size, 1])
# #         fake_code2 = np.random.normal(scale=0.5, size=[batch_size, 1])
#         inputs = concatenate([noise, fake_labels, fake_codes])
#         fake_images = generator.predict(inputs)

#         # real + fake images = 1 batch of train data
#         x = np.concatenate((real_images, fake_images))
#         labels = np.concatenate((real_labels, fake_labels))
#         codes = np.concatenate([(real_code, fake_code) for real_code, fake_code in zip(real_codes, fake_codes)])
# #         codes1 = np.concatenate((real_code1, fake_code1))
# #         codes2 = np.concatenate((real_code2, fake_code2))

#         # label real and fake images
#         # real images label is 1.0
#         y = np.ones([2 * batch_size, 1])
#         # fake images label is 0.0
#         y[batch_size:, :] = 0

#         # train discriminator network, log the loss and label accuracy
#         outputs = concatenate([y, labels, codes])
#         # metrics = ['loss', 'activation_1_loss', 'label_loss',
#         # 'code1_loss', 'code2_loss', 'activation_1_acc',
#         # 'label_acc', 'code1_acc', 'code2_acc']
#         # from discriminator.metrics_names
#         metrics = discriminator.train_on_batch(x, outputs)
#         fmt = "%d: [discriminator loss: %f, label_acc: %f]"
#         log = fmt % (i, metrics[0], metrics[6])

#         # train the adversarial network for 1 batch
#         # 1 batch of fake images with label=1.0 and
#         # corresponding one-hot label or class + random codes
#         # since the discriminator weights are frozen in adversarial network
#         # only the generator is trained
#         # generate fake images, labels and codes
#         noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
#         fake_labels = np.eye(num_labels)[np.random.choice(num_labels,
#                                                           batch_size)]
# #         fake_code1 = np.random.normal(scale=0.5, size=[batch_size, 1])
# #         fake_code2 = np.random.normal(scale=0.5, size=[batch_size, 1])
#         # label fake images as real
#         y = np.ones([batch_size, 1])

#         # train the adversarial network 
#         # note that unlike in discriminator training, 
#         # we do not save the fake images in a variable
#         # the fake images go to the discriminator input of the adversarial
#         # for classification
#         # log the loss and label accuracy
#         inputs = [noise, fake_labels] + [fake_code for fake_code in fake_codes]
#         outputs = [y, fake_labels] + [fake_code for fake_code in fake_codes]
#         metrics  = adversarial.train_on_batch(inputs, outputs)
#         fmt = "%s [adversarial loss: %f, label_acc: %f]"
#         log = fmt % (log, metrics[0], metrics[6])

#         print(log)
#         if (i + 1) % save_interval == 0:
#             if (i + 1) == train_steps:
#                 show = True
#             else:
#                 show = False
    
#             # plot generator images on a periodic basis
#             plot_images(generator,
#                             noise_input=noise_input,
#                             noise_label=noise_label,
#                             noise_codes=noise_codes,
#                             show=show,
#                             step=(i + 1),
#                             model_name=model_name)
   
#     # save the model after training the generator
#     # the trained generator can be reloaded for
#     # future MNIST digit generation
#     generator.save(model_name + ".h5")
def train(models, x_train, params):
    """Train the Discriminator and Adversarial Networks
    Alternately train Discriminator and Adversarial networks by batch.
    Discriminator is trained first with properly real and fake images.
    Adversarial is trained next with fake images pretending to be real
    Generate sample images per save_interval.
    # Arguments
        models (list): Generator, Discriminator, Adversarial models
        x_train (tensor): Train images
        params (list) : Networks parameters
    """
    # the GAN models
    generator, discriminator, adversarial = models
    # network parameters
    batch_size, latent_size, train_steps, model_name = params
    # the generator image is saved every 500 steps
    save_interval = 500
    # noise vector to see how the generator output evolves during training
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
    # number of elements in train dataset
    train_size = x_train.shape[0]
    for i in range(train_steps):
        # train the discriminator for 1 batch
        # 1 batch of real (label=1.0) and fake images (label=0.0)
        # randomly pick real images from dataset
        rand_indexes = np.random.randint(0, train_size, size=batch_size)
        real_images = x_train[rand_indexes]
        # generate fake images from noise using generator 
        # generate noise using uniform distribution
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
        # generate fake images
        fake_images = generator.predict(noise)
        # real + fake images = 1 batch of train data
        x = np.concatenate((real_images, fake_images))
        # label real and fake images
        # real images label is 1.0
        y = np.ones([2 * batch_size, 1])
        # fake images label is 0.0
        y[batch_size:, :] = 0.0
        # train discriminator network, log the loss and accuracy
        loss, acc = discriminator.train_on_batch(x, y)
        log = "%d: [discriminator loss: %f, acc: %f]" % (i, loss, acc)

        # train the adversarial network for 1 batch
        # 1 batch of fake images with label=1.0
        # since the discriminator weights are frozen in adversarial network
        # only the generator is trained
        # generate noise using uniform distribution
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
        # label fake images as real or 1.0
        y = np.ones([batch_size, 1])
        # train the adversarial network 
        # note that unlike in discriminator training, 
        # we do not save the fake images in a variable
        # the fake images go to the discriminator input of the adversarial
        # for classification
        # log the loss and accuracy
        loss, acc = adversarial.train_on_batch(noise, y)
        log = "%s [adversarial loss: %f, acc: %f]" % (log, loss, acc)
        print(log)
        if (i + 1) % save_interval == 0:
            if (i + 1) == train_steps:
                show = True
            else:
                show = False

            # plot generator images on a periodic basis
            plot_images(generator,
                        noise_input=noise_input,
                        show=show,
                        step=(i + 1),
                        model_name=model_name)
   
    # save the model after training the generator
    # the trained generator can be reloaded for future MNIST digit generation
    generator.save(model_name + ".h5")


def mi_loss(c, q_of_c_given_x):
    """ Mutual information, Equation 5 in [2] , assuming H(c) is constant"""
    # mi_loss = -c * log(Q(c|x))
    return K.mean(-K.sum(K.log(q_of_c_given_x + K.epsilon()) * c, axis=1))


def test_generator(generator, params, latent_size=100):
    label, code1, code2, p1, p2 = params
    noise_input = np.random.uniform(-1., 1., size=[16, latent_size])
    step = 0
    if label is None:
        num_labels = 13
        noise_label = np.eye(num_labels)[np.random.choice(num_labels, 16)]
    else:
        noise_label = np.zeros((16, 13))
        noise_label[:,label] = 1
        step = label

    if code1 is None:
        noise_code1 = np.ones((16, 1)) * -1
    else:
        if p1:
            a = np.linspace(-1, 1, 16)
            a = np.reshape(a, [16, 1])
            noise_code1 = np.ones((16, 1)) * a
        else:
            noise_code1 = np.ones((16, 1)) * code1
        print(noise_code1)

    if code2 is None:
        noise_code2 = np.ones((16, 1)) * -1
    else:
        if p2:
            a = np.linspace(-1, 1, 16)
            a = np.reshape(a, [16, 1])
            noise_code2 = np.ones((16, 1)) * a
        else:
            noise_code2 = np.ones((16, 1)) * code2
        print(noise_code2)

    plot_images(generator,
                    noise_input=noise_input,
                    noise_label=noise_label,
                    noise_codes=[noise_code1, noise_code2],
                    show=False,
                    step=step,
                    model_name="test_outputs_{}".format(label))


In [73]:
model_name = "infogan_butterflies_darknet_generator_weirdweights"

# load dataset
(x_train, y_train), (_, _) = butterflies.load_data(path='../data/raw/butterflies.npz')

# reshape data for CNN as (28, 28, 1) and normalize
image_size = x_train.shape[1]
x_train = np.reshape(x_train, [-1, image_size, image_size, 3])
x_train = x_train.astype('float32')


####
#plotting sanity check

os.makedirs(model_name, exist_ok=True)
filename = os.path.join(model_name, "example_real_inputs.png")
rows = int(math.sqrt(16))
images = x_train[0:16]
plt.figure(figsize=(10, 10))
num_images = 16
image_size = 256
for i in range(num_images):
    plt.subplot(rows, rows, i + 1)
    image = np.reshape(images[i], [image_size, image_size, 3])
    plt.imshow(image)
    plt.axis('off')
plt.savefig(filename)
plt.close('all')
####

# train labels
num_labels = len(np.unique(y_train))
num_codes = 10
print(num_labels)
_, y_train = np.unique(y_train,return_inverse=True)
y_train = to_categorical(y_train)

# network parameters
batch_size = 32
train_steps = 100000
lr = 2e-4
decay = 6e-8
input_shape = (image_size, image_size, 3)
label_shape = (num_labels, )
code_shape = (1, )
latent_size = 100

# build discriminator model
inputs = Input(shape=input_shape, name='discriminator_input')
# call discriminator builder with 4 outputs: source, label, and 2 codes
discriminator = discriminator(inputs,
                                  num_labels=num_labels,
                                  num_codes=num_codes)
print(discriminator.outputs,discriminator.inputs)
# [1] uses Adam, but discriminator converges easily with RMSprop
optimizer = RMSprop(lr=lr, decay=decay)
# loss functions: 1) probability image is real (binary crossentropy)
# 2) categorical cross entropy image label,
# 3) and 4) mutual information loss
loss = ['binary_crossentropy', 'categorical_crossentropy']
for i in range(num_codes):
    loss.append(mi_loss)
# lamda or mi_loss weight is 0.5
loss_weights = [1.0, 1.0] + [1.0 for i in range(num_codes)]
discriminator.compile(loss=loss,
                      loss_weights=loss_weights,
                      optimizer=optimizer,
                      metrics=['accuracy'])
discriminator.summary()

# build generator model
input_shape = (latent_size, )
inputs = Input(shape=input_shape, name='z_input')
labels = Input(shape=label_shape, name='labels')
codes = [Input(shape=code_shape, name="code{}".format(i)) for i in range(num_codes)]
# code1 = Input(shape=code_shape, name="code1")
# code2 = Input(shape=code_shape, name="code2")
# call generator with inputs, 
# labels and codes as total inputs to generator
generator = generator(inputs,
                          image_size,
                          labels=labels,
                          codes=codes)
generator.summary()

# build adversarial model = generator + discriminator
optimizer = RMSprop(lr=lr*0.5, decay=decay*0.5)
discriminator.trainable = False
# total inputs = noise code, labels, and codes
inputs = [inputs, labels] + codes
adversarial = Model(inputs,
                    discriminator(generator(inputs)),
                    name=model_name)
# same loss as discriminator
adversarial.compile(loss=loss,
                    loss_weights=loss_weights,
                    optimizer=optimizer,
                    metrics=['accuracy'])
adversarial.summary()

# train discriminator and adversarial networks
models = (generator, discriminator, adversarial)
data = (x_train, y_train)
params = (batch_size, latent_size, train_steps, num_labels, model_name)
train(models, data, params, num_codes)

8
sigmoid
[<tf.Tensor 'activation_35/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'label_7/Identity:0' shape=(None, 8) dtype=float32>, <tf.Tensor 'code0_14/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code1_14/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code2_14/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code3_14/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code4_14/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code5_14/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code6_14/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code7_14/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code8_14/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code9_14/Identity:0' shape=(None, 1) dtype=float32>] [<tf.Tensor 'discriminator_input_9:0' shape=(None, 256, 256, 3) dtype=float32>]
Model: "discriminator"
__________________________________________________________________________________________________
Lay

Model: "generator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
z_input (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
labels (InputLayer)             [(None, 8)]          0                                            
__________________________________________________________________________________________________
code0 (InputLayer)              [(None, 1)]          0                                            
__________________________________________________________________________________________________
code1 (InputLayer)              [(None, 1)]          0                                            
__________________________________________________________________________________________

TypeError: train() takes 3 positional arguments but 4 were given

In [ ]:
generator = load_model("infogan_butterflies_darknet_generator.h5")
label = 0
code1 = None
code2 = True
p1 = None
p2 = True
params = (label, code1, code2, p1, p2)
test_generator(generator, params, latent_size=100)